<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_06_Publish_Summary_of_Web_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, and then summarize the contents.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

async def create_feed(search_text: str, read_limit: int):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            text=search_text,
            readLimit=read_limit
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def summarize_feed(feed_id: str, prompt: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.summarize_contents(
            filter=input_types.ContentFilter(
                # Filter just on feed
                feeds=[input_types.EntityReferenceFilter(id=feed_id)],
            ),
            summarizations=[
                input_types.SummarizationStrategyInput(
                    type=enums.SummarizationTypes.CUSTOM,
                    prompt=prompt
                ),
            ]
        )

        return response.summarize_contents
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def publish_feeds(prompt: str, feeds: List[str]):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.publish_contents(
            name="Web Summary",
            connector=input_types.ContentPublishingConnectorInput(
               type=enums.ContentPublishingServiceTypes.TEXT,
               format=enums.ContentPublishingFormats.MARKDOWN
            ),
            publish_prompt = prompt,
            filter=input_types.ContentFilter(
                # Filter just on feeds
                feeds=[input_types.EntityReferenceFilter(id=feed_id) for feed_id in feeds],
            ),
            is_synchronous=True
        )

        return response.publish_contents.markdown if response.publish_contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [5]:
from IPython.display import display, Markdown
import time

# Remove any existing feeds; only needed for notebook example
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 3 # how many search results to ingest from each feed

searches = [
    "Where has Monkeypox been found?",
    "How does Monkeypox affect the body?",
    "What vaccines are being developed against Monkeypox?",
]

feeds = []

for search_text in searches:
    feed_id = await create_feed(search_text, read_limit)

    if feed_id is not None:
        print(f'Created feed [{feed_id}].')

        # Wait for feed to complete, since ingestion happens asychronously
        done = False
        time.sleep(5)
        while not done:
            done = await is_feed_done(feed_id)

            if not done:
                time.sleep(2)

        print(f'Completed feed [{feed_id}].')

        feeds.append(feed_id)

prompt = """
Read the text first, all the way to the end.
Condense the text into 5 verbose paragraphs.
Be specific when referencing persons, organizations, or any other named entities.
"""

print()
print('Summarizing feeds...')

# First, we will summarize the results of each web search, independently
for feed_id in feeds:
    summarizations = await summarize_feed(feed_id, prompt)

    if summarizations is not None:
        print(f'Summarized feed [{feed_id}]:')

        for summarization in summarizations:
            if summarization is not None and summarization.items is not None:
                for item in summarization.items:
                    display(Markdown(item.text))

publish_prompt = """
Follow these steps:
Step 1: Think about a structure for an engaging AI-generated blog post, with an introduction, an in-depth discussion of 4-6 interesting topics across all provided content, and a wrap-up.
Step 2: For each topic, write 2-3 detailed paragraphs discussing it in-depth. Touch on key points for each topic which would be interesting to readers. Mention the content metadata, entities and details from the provided summaries, as appropriate in the discussion.
Step 3: Combine all topics into a lengthy blog post. Use friendly but technical conversation to write the post.
Step 4: Remove any unnecessary formatting or final notes about being AI generated.
"""

print()
print('Publishing feeds...')

# Next, we will publish a blog post based on all the content, across all the web search feeds
published_markdown = await publish_feeds(publish_prompt, feeds)

if published_markdown is not None:
    display(Markdown(published_markdown))

Deleted all feeds.
Created feed [c9edc166-3725-4f47-846c-89e5a0655f0b].
Completed feed [c9edc166-3725-4f47-846c-89e5a0655f0b].
Created feed [907dfbff-f3f3-4b82-b61d-a8337e419db1].
Completed feed [907dfbff-f3f3-4b82-b61d-a8337e419db1].
Created feed [bb16f110-ace8-4af3-941e-f2b84ede0c99].
Completed feed [bb16f110-ace8-4af3-941e-f2b84ede0c99].

Summarizing feeds...
Summarized feed [c9edc166-3725-4f47-846c-89e5a0655f0b]:


Mpox, previously known as monkeypox, is a viral illness caused by the monkeypox virus, a member of the Orthopoxvirus genus. The virus has two distinct clades: clade I (with subclades Ia and Ib) and clade II (with subclades IIa and IIb). A global outbreak of mpox caused by the clade IIb strain occurred in 2022-2023, and the disease remains a threat today. Recent upsurges in cases in the Democratic Republic of the Congo and other countries have been attributed to clades Ia and Ib. Mpox symptoms include a skin rash or mucosal lesions, fever, headache, muscle aches, back pain, low energy, and swollen lymph nodes. The disease can be transmitted through close contact with infected individuals, contaminated materials, or infected animals. Vaccination is available and should be considered alongside other public health interventions.

Mpox spreads primarily through close contact with an infected person, including skin-to-skin, mouth-to-mouth, or mouth-to-skin contact. People with multiple sexual partners are at higher risk. The virus can also be contracted from contaminated objects, needle injuries, or during pregnancy and birth. Animal-to-human transmission occurs through bites, scratches, or activities like hunting and cooking infected animals. Symptoms usually begin within a week of exposure and last 2-4 weeks. Common symptoms include rash, fever, sore throat, headache, muscle aches, back pain, low energy, and swollen lymph nodes. The rash often starts on the face and spreads to other body parts, including the palms, soles, and genitals.

Diagnosing mpox can be challenging due to its similarity to other infections and conditions. The preferred diagnostic method is polymerase chain reaction (PCR) testing of samples taken from the rash. HIV testing should be offered to adults with mpox, and diagnostic tests for other conditions like syphilis and herpes should be considered. Treatment focuses on managing symptoms and preventing complications. Vaccination can help prevent infection and is recommended for high-risk groups, including health workers, people with multiple sex partners, and sex workers. Some antivirals have received emergency use authorization, but no proven effective antiviral treatment exists yet.

Preventive measures include isolating at home, washing hands frequently, wearing masks, and covering lesions. People with mpox should avoid touching shared items and disinfect shared spaces. Sexual activity should be avoided during periods of increased transmission, and condoms should be used as a precaution for 12 weeks after recovery. Health workers should follow infection prevention protocols, including wearing personal protective equipment (PPE). The first human case of mpox was reported in the Democratic Republic of the Congo in 1970. Since then, the disease has spread sporadically in central and east Africa and more recently in other regions, including Europe and the Americas.

Stigma and discrimination related to mpox, particularly against men who have sex with men, trans people, and gender-diverse communities, can undermine public health efforts. The World Health Organization (WHO) works with member states and partners to prevent and respond to mpox outbreaks. This includes coordinating research on vaccines and treatments, strengthening health systems, and ensuring equitable access to medical tools. WHO Director-General Dr. Tedros Adhanom Ghebreyesus has declared mpox a public health emergency of international concern twice, first in May 2022 and again in August 2024.

As of June 2, 2022, the Centers for Disease Control and Prevention (CDC) confirmed 19 cases of monkeypox across 10 U.S. states. The first case was reported in Massachusetts on May 18, and recent cases have emerged in Florida, Georgia, Pennsylvania, and New York. There are also suspected cases in California and Illinois awaiting confirmation. CDC Director Dr. Rochelle Walensky emphasized the need to presume community spread, noting that the U.S. has been preparing for such outbreaks for decades. The U.S. has two vaccines approved by the FDA for smallpox, with one, Jynneos, also approved for monkeypox. Some doses have already been distributed to close contacts of confirmed cases.

The majority of the cases have been among men who have sex with men, but Dr. Walensky stressed that the risk of exposure is not limited to any particular group. Dr. Raj Panjabi from the White House National Security Council echoed this sentiment, urging vigilance across all communities. The first U.S. case involved a man hospitalized in Massachusetts, whose contacts, including healthcare workers, were offered vaccines. Since then, cases have been confirmed in several states, with some linked to international travel. For instance, Virginia's case involved a woman who had traveled to Africa, and California's cases were linked to travel in Europe.

The CDC has confirmed that all U.S. patients have contracted the West African strain of monkeypox, which is generally mild with a low fatality rate. The U.S. case count is relatively small compared to European countries like Spain and England, which have reported over 150 and 200 cases, respectively. Globally, more than 770 cases have been confirmed outside Africa since the outbreak began in May. Monkeypox is endemic in 11 African countries, with most cases historically linked to animal exposure. However, current transmission appears to be through skin-to-skin contact among people.

Experts are optimistic about containing the outbreak through contact tracing and targeted vaccinations. The Jynneos vaccine, approved in 2019, is the only one specifically for monkeypox, but older smallpox vaccines also offer significant protection. Jennifer McQuiston from the CDC's High Consequence Pathogens and Pathology division stated that efforts are focused on containing current cases to prevent further spread. The CDC is also considering whether broader vaccination recommendations might be necessary.

In summary, the U.S. is actively responding to the monkeypox outbreak with targeted vaccinations and contact tracing. While the majority of cases have been among men who have sex with men, the risk is not confined to any single group. The West African strain involved is generally mild, and experts believe the outbreak can be contained. The CDC continues to monitor the situation and may expand vaccination efforts if needed.

NBC News is actively tracking the spread of mpox (formerly known as monkeypox) across the United States, updating the data weekly. The World Health Organization (WHO) declared mpox an international public health emergency in July, and the virus has been recorded in every U.S. state. Initially, cases were reported in 14 states and Washington, D.C., in early June, which expanded to 44 states and Washington, D.C., by the end of July. As of now, the U.S. has reported over 29,000 cases since May, contributing to a global total exceeding 80,000 cases.

The spread of mpox has seen a significant decline, with the seven-day average of new cases dropping from 457 on August 6 to fewer than 5 by January 5. This decrease marks a substantial improvement from the peak in August when more than 400 cases were reported daily. The virus primarily spreads through close physical contact, and recent cases have predominantly been among men who have sex with men.

In response to the outbreak, several states have taken emergency measures. California, Illinois, and New York state issued emergency declarations to combat the continued spread of mpox. These declarations followed the WHO's global emergency status and aimed to mobilize resources and raise public awareness about the virus.

The World Health Organization renamed monkeypox to mpox in November to address concerns about racism associated with the original name. This change reflects a broader effort to use non-stigmatizing language in public health communications. The Centers for Disease Control and Prevention (CDC) continues to monitor and report on the situation, providing data that NBC News uses for its weekly updates.

Joe Murphy, a data editor, and Alex Ford, an interactive journalist at NBC News, are responsible for compiling and analyzing the data on mpox cases. Their work ensures that the public remains informed about the status of the outbreak. Dana Varinsky also contributed to the reporting, highlighting the collaborative effort to track and report on this public health issue.

Summarized feed [907dfbff-f3f3-4b82-b61d-a8337e419db1]:


...

Summarized feed [bb16f110-ace8-4af3-941e-f2b84ede0c99]:


The global outbreak of monkeypox has resulted in over 24,000 cases across more than 80 countries, prompting the World Health Organization (WHO) to warn that the opportunity to contain the disease is rapidly closing. Vaccines are seen as a crucial measure to combat the spread, but there are significant challenges in terms of availability and efficacy. Three vaccines, originally developed for smallpox, are being considered: MVA-BN (Jynneos in the US), ACAM2000, and LC16m8. Of these, only Jynneos is licensed for monkeypox by the US Food and Drug Administration (FDA). The scramble for these limited doses has raised concerns about equitable access, especially for poorer nations.

Jynneos is considered the safest of the three vaccines, as it contains a non-replicating form of the vaccinia virus, reducing the risk of illness. In contrast, ACAM2000 and LC16m8 contain replicating forms of the virus, which can potentially make people unwell, particularly those with weakened immune systems. The administration methods also differ: Jynneos is given via injection under the skin in two doses, while ACAM2000 is delivered through a needle that scratches the skin, creating a wound that can shed the virus. This makes Jynneos the preferred choice in most regions.

The cessation of routine smallpox vaccination in the 1970s may have contributed to the current monkeypox outbreak. Both smallpox and monkeypox are poxviruses, sharing about 85% genetic similarity. Historically, smallpox vaccines have provided protection against other poxviruses, including monkeypox. However, the effectiveness of these vaccines against monkeypox in humans remains uncertain. Jynneos has shown promising immune responses in animal models, but it has not been tested in controlled human trials due to the sporadic nature of past outbreaks.

The global supply of monkeypox vaccines is limited, with the WHO estimating around 16 million doses available, most of which are in bulk form requiring further processing. Bavarian Nordic, the manufacturer of Jynneos, has a significant portion of its stock owned by the US, which contracted the company to develop a safer smallpox vaccine in the early 2000s. Despite concerns about supply bottlenecks, Bavarian Nordic claims to have met all requests so far and has scaled up production to meet increasing demand.

Equitable distribution of the available vaccines remains a critical issue. The WHO has urged countries with stockpiles to share doses with those lacking access. The US Strategic National Stockpile includes millions of doses of ACAM2000 and thousands of Jynneos doses. However, the vaccines were initially developed for stockpiling against potential bioterrorism threats, complicating the current distribution efforts. Ensuring that these vaccines reach the countries most in need, particularly those in Africa where monkeypox is endemic, is essential for controlling the outbreak.

The JYNNEOS™ vaccine is designed to protect against smallpox, monkeypox, and other diseases caused by orthopoxviruses, including the vaccinia virus. Smallpox, caused by the variola virus, is a severe disease with a high mortality rate, historically killing about 30% of those infected. Although smallpox has been eradicated, certain individuals, such as emergency preparedness workers and laboratory personnel, remain at risk of exposure. Monkeypox, while rarer and generally less severe than smallpox, can still be fatal and has seen outbreaks in Africa and imported cases in other countries, including the United States. The vaccinia virus, used in some vaccines, can also cause disease, particularly in laboratory settings, but these infections typically resolve without treatment.

The JYNNEOS™ vaccine, which uses a weakened live vaccinia virus, is FDA-approved for adults 18 years and older who are at high risk for smallpox or monkeypox infection. The CDC recommends this vaccine for specific groups, including certain laboratory workers and emergency response team members. It is administered in a series of two injections, spaced four weeks apart, with booster doses recommended every 2 to 10 years for those at continued risk. Individuals who have previously received a smallpox vaccine may only need one dose. The vaccine is generally safe to administer alongside other vaccines, though certain individuals at risk for myocarditis may need to wait before receiving certain COVID-19 vaccines.

Before receiving the JYNNEOS™ vaccine, individuals should inform their healthcare provider of any severe allergies, immune system issues, or if they are pregnant or breastfeeding. In some cases, vaccination may be postponed for those with minor illnesses, but those exposed to the monkeypox virus should be vaccinated regardless of other health conditions. Common side effects include redness, soreness, swelling, and itching at the injection site, as well as fatigue, headache, and muscle pain. Severe allergic reactions are rare but possible, and any concerning symptoms should be reported to a healthcare provider or emergency services.

In the event of a serious reaction, individuals should seek immediate medical attention and report the incident to the Vaccine Adverse Event Reporting System (VAERS). The Countermeasures Injury Compensation Program may provide financial assistance for medical care and other expenses for those seriously injured by the vaccine. More information about this program can be found on its website or by calling the provided hotline.

For further information, individuals are encouraged to consult their healthcare provider, contact local or state health departments, or visit the CDC and FDA websites. The CDC also offers resources in multiple languages, including Spanish, to ensure accessibility for diverse populations. The JYNNEOS™ vaccine information statement was last reviewed on November 14, 2022, by the National Center for Immunization and Respiratory Diseases.


Publishing feeds...


# Understanding Mpox: A Comprehensive Guide

## Introduction

Mpox, formerly known as monkeypox, has emerged as a significant public health concern globally. This viral illness, caused by the monkeypox virus, has seen outbreaks in various parts of the world, prompting health organizations to take action. In this blog post, we will delve into the intricacies of mpox, exploring its transmission, symptoms, diagnosis, treatment, and prevention. We will also discuss the global response to the outbreak and the role of vaccines in controlling the spread of the virus.

## The Nature of Mpox

Mpox is a viral illness caused by the monkeypox virus, a member of the Orthopoxvirus genus. There are two distinct clades of the virus: clade I and clade II, with the latter being responsible for the global outbreak that began in 2022. The virus spreads through close contact with an infected person, contaminated materials, or infected animals. This includes skin-to-skin, mouth-to-mouth, or mouth-to-skin contact, and can also be transmitted during pregnancy to the fetus or to the newborn during or after birth. People with multiple sexual partners are at higher risk, and animal-to-human transmission can occur through bites, scratches, or activities like hunting and cooking infected animals.

The symptoms of mpox include a skin rash, fever, headache, muscle aches, back pain, low energy, and swollen lymph nodes. The rash often begins on the face and spreads to other parts of the body, including the palms and soles. Symptoms typically last 2–4 weeks but may last longer in individuals with weakened immune systems. Complications can include bacterial skin infections, pneumonia, corneal infection, and sepsis. Diagnosis is challenging due to similarities with other infections and conditions, with the preferred test being polymerase chain reaction (PCR) to detect viral DNA from skin lesions.

## Mpox in the United States

The Centers for Disease Control and Prevention (CDC) confirmed 19 cases of mpox across 10 U.S. states as of June 2, 2022. The first confirmed case was reported in Massachusetts on May 18, 2022. Recent cases have been confirmed in Florida, Georgia, Pennsylvania, and New York, with suspected cases in California and Illinois awaiting confirmation. Some U.S. patients had traveled to areas where mpox is spreading, but not all, indicating potential community spread. The CDC has resources to respond to the outbreak, including two vaccines approved by the FDA for smallpox, with one also approved for mpox (Jynneos).

Most cases are among men who have sex with men, but the risk of exposure is not limited to any particular group. The first U.S. case involved a man hospitalized in Massachusetts on May 12, who remains in good condition. Massachusetts health officials are investigating over 200 of his contacts, mostly healthcare workers. The CDC confirmed additional cases in New York, California, Florida, Colorado, Utah, Georgia, Pennsylvania, Virginia, and Washington. The U.S. patients have the West African strain of mpox, which tends to cause mild disease with a 1% fatality rate.

## Tracking the Outbreak

NBC News is tracking U.S. cases of mpox, which the World Health Organization labeled an international “public health emergency.” The data is updated weekly. Mpox infections have significantly decreased, with fewer than 5 cases a day as of January 5, 2023, down from a peak of more than 400 cases a day in August 2022. Cases have been recorded in every state, increasing from 44 states and Washington, D.C., at the end of July, and from 14 states and Washington, D.C., at the beginning of June. The U.S. has reported more than 29,000 cases since May, while the total worldwide cases exceed 80,000.

The virus is primarily transmitted through close physical contact, with recent cases mostly among men who have sex with men. The World Health Organization declared the outbreak a public health emergency in July. California, Illinois, and New York state have issued emergency declarations over the continued spread of mpox in the last three weeks. The seven-day average of reported new cases decreased from 457 on August 6 to fewer than 5 on January 5, according to an NBC News analysis of CDC data.

## Vaccination and Treatment

The JYNNEOS vaccine is designed to protect against smallpox, mpox, and other diseases caused by orthopoxviruses, including the vaccinia virus. Approved by the FDA for adults 18 years or older at high risk for smallpox or mpox infection, it is administered as a series of 2 injections, 4 weeks apart, with booster doses recommended every 2 or 10 years for those at continued risk. Common reactions include redness, soreness, swelling, itching, fatigue, headache, and muscle pain. Severe allergic reactions are rare but possible, and emergency medical attention should be sought if they occur.

The ACAM2000 vaccine contains a live virus that can multiply and is administered via a two-pronged needle prick on the upper arm. It can cause a lesion at the injection site, which takes about 6 weeks to heal. Side effects include fever, rash, lymph node swelling, myocarditis, and pericarditis. The vaccine is not recommended for individuals with weak immune systems. Post-exposure prophylaxis (PEP) is effective if given within 4 days of exposure and can still be beneficial if given within 4-14 days post-exposure, though less effective. Pre-exposure prophylaxis (PrEP) is recommended for high-risk individuals.

## Global Response and Challenges

The global outbreak of mpox has resulted in over 24,000 cases across more than 80 countries. The World Health Organization (WHO) has indicated that the opportunity to contain the disease and prevent it from becoming endemic outside Africa is diminishing. Three vaccines are being considered to combat the outbreak, all originally developed for smallpox. The vaccines are MVA-BN (Jynneos in the US), ACAM2000, and LC16m8. The effectiveness of these vaccines against mpox is not fully known, and historical data suggests smallpox vaccines provided long-lasting protection against mpox.

There are concerns about equitable access to vaccines, especially for poorer nations. The WHO has called for nations with vaccine stockpiles to share doses with those that do not have them. The US Strategic National Stockpile includes millions of doses of ACAM2000 and thousands of doses of Jynneos. The end of widespread smallpox vaccination in the 1970s may have contributed to the current mpox outbreak. The global population under 50 years old is more susceptible to poxvirus infections due to the cessation of smallpox vaccinations.

## Conclusion

Mpox remains a significant public health challenge, but with coordinated efforts, it is possible to control its spread. Vaccination, public awareness, and preventive measures are crucial in managing the outbreak. Health organizations worldwide continue to monitor the situation and provide resources to mitigate the impact of the virus. By staying informed and taking appropriate precautions, we can contribute to the global effort to combat mpox.